In [1]:
 # Make data directory if it doesn't exist
!mkdir -p dataac
!wget -nc https://s3.amazonaws.com/assets.datacamp.com/production/course_3629/fake_or_real_news.csv -P data

--2021-06-09 04:59:28--  https://s3.amazonaws.com/assets.datacamp.com/production/course_3629/fake_or_real_news.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.14.198
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.14.198|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30696129 (29M) [text/csv]
Saving to: ‘data/fake_or_real_news.csv’

fake_or_real_news.c 100%[===================>]  29.27M  9.70MB/s    in 3.0s    

2021-06-09 04:59:32 (9.70 MB/s) - ‘data/fake_or_real_news.csv’ saved [30696129/30696129]



In [3]:
import pandas as pd

news = pd.read_csv("data/fake_or_real_news.csv")
news

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
X=news['title']
y=news['label']

#Text Preprocessing

In [6]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
porter = PorterStemmer()
corpus = []

for x in X:
  te = re.sub('[^A-Za-z]',' ',x)
  te = te.lower()
  te = te.split()
  te = [porter.stem(t) for t in te if t not in stopwords.words('english')]
  te = ' '.join(te)
  corpus.append(te)

In [8]:
print(X[1])
print(corpus[1])

Watch The Exact Moment Paul Ryan Committed Political Suicide At A Trump Rally (VIDEO)
watch exact moment paul ryan commit polit suicid trump ralli video


Count Vectorizer

In [9]:
count_vectorizer = CountVectorizer(max_features=5000,ngram_range=(1,3))
count_train = count_vectorizer.fit_transform(corpus)

In [10]:
count_df = pd.DataFrame(count_train.A,columns=count_vectorizer.get_feature_names())
print(count_df.head())

   abandon  abc  abc news  abdullah  ...  zionist stoog  zone  zuckerberg  zuess
0        0    0         0         0  ...              0     0           0      0
1        0    0         0         0  ...              0     0           0      0
2        0    0         0         0  ...              0     0           0      0
3        0    0         0         0  ...              0     0           0      0
4        0    0         0         0  ...              0     0           0      0

[5 rows x 5000 columns]


TfidfVectorizer

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vector = TfidfVectorizer(max_features=5000,ngram_range=(1,3))

tfidf_train = tfidf_vector.fit_transform(corpus)

In [12]:
tfidf_df = pd.DataFrame(tfidf_train.A,columns=tfidf_vector.get_feature_names())
print(tfidf_df.head())

   abandon  abc  abc news  abdullah  ...  zionist stoog  zone  zuckerberg  zuess
0      0.0  0.0       0.0       0.0  ...            0.0   0.0         0.0    0.0
1      0.0  0.0       0.0       0.0  ...            0.0   0.0         0.0    0.0
2      0.0  0.0       0.0       0.0  ...            0.0   0.0         0.0    0.0
3      0.0  0.0       0.0       0.0  ...            0.0   0.0         0.0    0.0
4      0.0  0.0       0.0       0.0  ...            0.0   0.0         0.0    0.0

[5 rows x 5000 columns]


In [13]:
X_train_count,X_test_count,y_train_count,y_test_count = train_test_split(count_df,y,test_size=0.3,random_state=52)
X_train_tfidf,X_test_tfidf,y_train_tfidf,y_test_tfidf = train_test_split(tfidf_df,y,test_size=0.3,random_state=52)

#Machine learning

Naive bayes to classify 

In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_count,y_train_count)
pred = nb_classifier.predict(X_test_count)
print(metrics.accuracy_score(y_test_count,pred))
print(metrics.confusion_matrix(y_test_count,pred))

0.820620725933719
[[765 187]
 [154 795]]


In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf,y_train_tfidf)
pred = nb_classifier.predict(X_test_tfidf)
print(metrics.accuracy_score(y_test_tfidf,pred))
print(metrics.confusion_matrix(y_test_tfidf,pred))

0.8169384534455549
[[762 190]
 [158 791]]


In [16]:
#Most real
sorted(zip(nb_classifier.coef_[0],tfidf_vector.get_feature_names()),reverse=True)[:5]

[(-4.961405609574252, 'trump'),
 (-5.360943493023131, 'clinton'),
 (-5.478616663823097, 'obama'),
 (-5.548883838553669, 'gop'),
 (-5.7974917758159235, 'republican')]

In [ ]:
#Most real
sorted(zip(nb_classifier.coef_[0],tfidf_vector.get_feature_names()),reverse=False)[:5]

[(-9.296974057971962, 'abc news'),
 (-9.296974057971962, 'abedin weiner'),
 (-9.296974057971962, 'abedin weiner laptop'),
 (-9.296974057971962, 'abil'),
 (-9.296974057971962, 'abstain')]

In [17]:
from sklearn.linear_model import PassiveAggressiveClassifier
linear_clf = PassiveAggressiveClassifier(n_iter_no_change=50)

linear_clf.fit(X_train_count,y_train_count)
pred = linear_clf.predict(X_test_count)
print(metrics.accuracy_score(y_test_count,pred))
print(metrics.confusion_matrix(y_test_count,pred))

0.7601262493424513
[[712 240]
 [216 733]]


In [18]:
from sklearn.linear_model import PassiveAggressiveClassifier
linear_clf = PassiveAggressiveClassifier(n_iter_no_change=50)

linear_clf.fit(X_train_tfidf,y_train_tfidf)
pred = linear_clf.predict(X_test_tfidf)
print(metrics.accuracy_score(y_test_tfidf,pred))
print(metrics.confusion_matrix(y_test_tfidf,pred))

0.7659126775381379
[[724 228]
 [217 732]]


#LSTM

In [19]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
voc_size = 5000

In [20]:
onehot_rep = [one_hot(words,voc_size) for words in corpus]
print(onehot_rep[:4])

[[612, 4728, 4952], [2490, 3106, 1168, 832, 1846, 1866, 3423, 1425, 2203, 408, 1529], [1007, 2435, 3783, 1107, 4577], [1765, 1008, 2182, 1061, 1902, 3493, 1919, 565]]


In [21]:
#sent_length = 20
#embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)

embedded_docs = pad_sequences(onehot_rep,padding='pre')
print(embedded_docs.shape)
print(embedded_docs)
sent_length = embedded_docs.shape[1]

(6335, 26)
[[   0    0    0 ...  612 4728 4952]
 [   0    0    0 ... 2203  408 1529]
 [   0    0    0 ... 3783 1107 4577]
 ...
 [   0    0    0 ... 2680  533 3583]
 [   0    0    0 ... 2220  826 3746]
 [   0    0    0 ... 1326 2203 1541]]


In [22]:
embedding_veactor_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_veactor_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 26, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 26, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
import numpy as np 
X_final = np.array(embedded_docs)
y_final = np.array(pd.get_dummies(y).iloc[:,1])

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_final,y_final)

In [25]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=64)

Epoch 1/50
75/75 [==============================] - 8s 13ms/step - loss: 0.6573 - accuracy: 0.6188 - val_loss: 0.5356 - val_accuracy: 0.7607
Epoch 2/50
75/75 [==============================] - 1s 7ms/step - loss: 0.4489 - accuracy: 0.8076 - val_loss: 0.4620 - val_accuracy: 0.7765
Epoch 3/50
75/75 [==============================] - 1s 7ms/step - loss: 0.3323 - accuracy: 0.8588 - val_loss: 0.4542 - val_accuracy: 0.7917
Epoch 4/50
75/75 [==============================] - 1s 7ms/step - loss: 0.2664 - accuracy: 0.8916 - val_loss: 0.5030 - val_accuracy: 0.7809
Epoch 5/50
75/75 [==============================] - 1s 7ms/step - loss: 0.2159 - accuracy: 0.9152 - val_loss: 0.5597 - val_accuracy: 0.7879
Epoch 6/50
75/75 [==============================] - 1s 8ms/step - loss: 0.1806 - accuracy: 0.9299 - val_loss: 0.5868 - val_accuracy: 0.7835
Epoch 7/50
75/75 [==============================] - 1s 7ms/step - loss: 0.1549 - accuracy: 0.9425 - val_loss: 0.6392 - val_accuracy: 0.7822
Epoch 8/50
75/75 [=

In [30]:
pred = model.predict_classes(X_test)
print(metrics.accuracy_score(y_test,pred))
print(metrics.confusion_matrix(y_test,pred))

0.7626262626262627
[[614 170]
 [206 594]]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


#Bi Directional LSTM

In [35]:
from tensorflow.keras.layers import LSTM, Bidirectional, Dropout, Dense, Embedding
from tensorflow.keras.models import Sequential

model1 = Sequential()
model1.add(Embedding(voc_size,embedding_veactor_features,input_length=sent_length))
model1.add(Dropout(0.3))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))

model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [36]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=64)

Epoch 1/50
75/75 [==============================] - 4s 18ms/step - loss: 0.6720 - accuracy: 0.5952 - val_loss: 0.6025 - val_accuracy: 0.7771
Epoch 2/50
75/75 [==============================] - 1s 10ms/step - loss: 0.4379 - accuracy: 0.8106 - val_loss: 0.4353 - val_accuracy: 0.7879
Epoch 3/50
75/75 [==============================] - 1s 10ms/step - loss: 0.2964 - accuracy: 0.8771 - val_loss: 0.4633 - val_accuracy: 0.7854
Epoch 4/50
75/75 [==============================] - 1s 10ms/step - loss: 0.2329 - accuracy: 0.9131 - val_loss: 0.5294 - val_accuracy: 0.7765
Epoch 5/50
75/75 [==============================] - 1s 10ms/step - loss: 0.1867 - accuracy: 0.9316 - val_loss: 0.5483 - val_accuracy: 0.7809
Epoch 6/50
75/75 [==============================] - 1s 10ms/step - loss: 0.1493 - accuracy: 0.9484 - val_loss: 0.5733 - val_accuracy: 0.7841
Epoch 7/50
75/75 [==============================] - 1s 11ms/step - loss: 0.1271 - accuracy: 0.9550 - val_loss: 0.6075 - val_accuracy: 0.7797
Epoch 8/50
75

In [37]:
pred = model1.predict_classes(X_test)
print(metrics.accuracy_score(y_test,pred))
print(metrics.confusion_matrix(y_test,pred))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.7531565656565656
[[580 204]
 [187 613]]
